# Atividade Somativa 2

Para essa atividade foi escolhido o dataset **seoul_bike_data**. Ele contém dados horários sobre o aluguel de bicicletas públicas em Seul, capital da Coreia do Sul, cobrindo um período de um ano (de 1º de dezembro de 2017 a 30 de novembro de 2018). 

## Visão Geral do Dataset
O objetivo principal ao usar este dataset é prever a demanda por bicicletas alugadas (Rented Bike Count) com base nas outras variáveis disponíveis.

# 1. Carregando o Dataset

Vamos carregar o dataset em um dataframe e analisar seus features.

O dataset é composto por 8.760 linhas (24 horas por dia × 365 dias) e 14 colunas. As colunas são:

| Nome da Coluna | Descrição | Tipo de Dado | Exemplo |
| :--- | :--- | :--- | :--- |
| **DateTime** | A data e hora do registro. | Objeto/Texto | 01/12/20172017-01-12 00:00:00 |
| **Day** | Dia do registro. | Numérico (Inteiro) | 01|
| **Weekday** | Dia da semana. | Numérico (Inteiro) | 7 |
| **Hour** | A hora do dia (0 a 23). | Numérico (Inteiro) | 0, 1, 2, ... |
| **Rented Bike Count** | **(Variável Alvo)** O número de bicicletas alugadas em uma determinada hora. | Numérico (Inteiro) | 254 |
| **Temperature(°C)** | A temperatura em graus Celsius. | Numérico (Float) | -5.2 |
| **Humidity(%)** | A umidade relativa do ar em porcentagem. | Numérico (Inteiro) | 37 |
| **Wind speed (m/s)** | A velocidade do vento em metros por segundo. | Numérico (Float) | 2.2 |
| **Visibility (10m)** | A visibilidade em uma escala de 10 metros (ex: 2000 significa 20km). | Numérico (Inteiro) | 2000 |
| **Dew point temp(°C)** | A temperatura do ponto de orvalho em graus Celsius. | Numérico (Float) | -17.6 |
| **Solar Radiation (MJ/m2)**| A radiação solar. Geralmente é zero durante a noite. | Numérico (Float) | 0.0 |
| **Rainfall(mm)** | A quantidade de chuva em milímetros. | Numérico (Float) | 0.0 |
| **Snowfall (cm)** | A quantidade de neve em centímetros. | Numérico (Float) | 0.0 |

## Bibliotecas 
Vamos começar importando as blibliotecas

In [1]:
import pandas as pd # importando o pandas para manipularmos o dataset
from ydata_profiling import ProfileReport # importando o pandas-profiling para fazer o profile do dataset
from sklearn.model_selection import train_test_split # função para dividir o dataset em treino e teste
import xgboost as xgb  # XGBoost para regressão
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler # importando somente o StandardScaler do scikit-learn
from sklearn.pipeline import Pipeline # utilizado para criar pipelines
from sklearn.metrics import * # importando todas as funções de métricas do scikit-learn
from sklearn.feature_selection import * # importando todas as funções específicas de seleção de atributos do scikit-learn

from sklearn import set_config # utilizado para mostrar os passos do pipeline de forma visual
set_config(display='diagram') # forçando para que os passos do pipeline sejam mostrados em visual

# forçando a saída do pandas para que somente 3 dígitos sejam mostrados no lugar da notação científica
pd.options.display.float_format = '{:.3f}'.format

In [2]:
%matplotlib inline

Agora vamos importar o dataset para um dataframe

In [3]:
df_bike = pd.read_excel('seoul_bike_data.xlsx')
    
print("Amostra dos dados (primeiras 5 linhas):")
display(df_bike.head())

print("\nInformações gerais sobre o DataFrame:")
df_bike.info()

#profile = ProfileReport(df_bike)
#profile.to_file("relatorio.html")

Amostra dos dados (primeiras 5 linhas):


,DateTime,Day,Weekday,Hour,Rented Bike Count,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
0,2017-01-12 00:00:00,12,5,0,254,-5.200,37,2.200,2000,-17.600,0.000,0.000,0.000
1,2017-01-12 01:00:00,12,5,1,204,-5.500,38,0.800,2000,-17.600,0.000,0.000,0.000
2,2017-01-12 02:00:00,12,5,2,173,-6.000,39,1.000,2000,-17.700,0.000,0.000,0.000
3,2017-01-12 03:00:00,12,5,3,107,-6.200,40,0.900,2000,-17.600,0.000,0.000,0.000
4,2017-01-12 04:00:00,12,5,4,78,-6.000,36,2.300,2000,-18.600,0.000,0.000,0.000



Informações gerais sobre o DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DateTime                   8760 non-null   object 
 1   Day                        8760 non-null   int64  
 2   Weekday                    8760 non-null   int64  
 3   Hour                       8760 non-null   int64  
 4   Rented Bike Count          8760 non-null   int64  
 5   Temperature(°C)            8760 non-null   float64
 6   Humidity(%)                8760 non-null   int64  
 7   Wind speed (m/s)           8760 non-null   float64
 8   Visibility (10m)           8760 non-null   int64  
 9   Dew point temperature(°C)  8760 non-null   float64
 10  Solar Radiation (MJ/m2)    8760 non-null   float64
 11  Rainfall(mm)               8760 non-null   float64
 12  Snowfall (cm)              8760 non-null   float64
dtypes: float6

# 2. Limpeza do dataset
Ao analisando a tabela de correlação estre as variáveis com a ferramenta **ydata_profiling**, foi possível notar o seguinte:

## Análise de Redundância (Multicolinearidade)

Este é o primeiro passo: encontrar atributos que medem quase a mesma coisa.

*   **`Temperature(°C)` vs. `Dew point temperature(°C)`: Correlação de 0.912**
    *   **Análise:** Este é um valor de correlação **extremamente alto**. Como esperado, a temperatura e a temperatura do ponto de orvalho estão fortemente ligadas. Manter ambas no modelo é redundante e pode confundir alguns algoritmos.
    *   **Ação Recomendada:** **remover uma dessas duas colunas**. Mas qual? Para decidir, olhamos a correlação de cada uma com a nossa variável alvo, `Rented Bike Count`.
        *   Correlação de `Temperature` com `Rented Bike Count` = **0.565**
        *   Correlação de `Dew point temperature` com `Rented Bike Count` = **0.374**
    *   **Decisão:** A `Temperature(°C)` tem uma correlação muito mais forte com o número de bicicletas alugadas. Portanto, **mantenha `Temperature(°C)` e removemos `Dew point temperature(°C)`**.

## Análise do Poder Preditivo (Relação com `Rented Bike Count`)

Agora, vamos focar na linha/coluna `Rented Bike Count` para ver quais atributos são os melhores preditores.

**Correlações Positivas (Quando estes aumentam, a demanda tende a aumentar):**

*   **`Temperature(°C)` (0.565):** Esta é a **correlação mais forte** com a variável alvo. Confirma a intuição de que a demanda por bicicletas aumenta significativamente com a temperatura. Este é o seu preditor mais importante.
*   **`Hour` (0.389):** Uma correlação positiva forte. Isso indica que, ao longo do dia (das 0h às 23h), há uma tendência geral de aumento na demanda, provavelmente devido aos picos da tarde/noite. A hora do dia é um preditor crucial.
*   **`Solar Radiation (MJ/m2)` (0.382):** Também uma correlação forte. Mais radiação solar significa um dia ensolarado e agradável, o que incentiva o uso de bicicletas.
*   **`Visibility (10m)` (0.176):** Correlação positiva, mas mais fraca. Melhor visibilidade (menos neblina) está associada a um leve aumento no número de aluguéis.
*   **`Wind speed (m/s)` (0.148):** Correlação positiva fraca. Isso é um pouco contraintuitivo, pois esperaríamos que ventos fortes desincentivassem o ciclismo. Essa correlação fraca pode indicar uma relação não-linear (ex: um pouco de vento é agradável, mas muito vento é ruim).

**Correlações Negativas (Quando estes aumentam, a demanda tende a diminuir):**

*   **`Rainfall(mm)` (-0.282):** Uma correlação negativa moderada. Como esperado, quando chove, as pessoas alugam menos bicicletas.
*   **`Snowfall (cm)` (-0.221):** Correlação negativa moderada. Similar à chuva, a neve também desincentiva o uso de bicicletas.
*   **`Humidity(%)` (-0.221):** Correlação negativa moderada. Alta umidade (clima abafado ou úmido) está associada a uma menor demanda.

**Correlações Próximas de Zero (Pouco Poder Preditivo Linear):**

*   **`Day` (0.067)** e **`Weekday` (0.062):** Essas colunas têm uma correlação linear muito fraca com a demanda. Isso **não significa que elas são inúteis!** Significa apenas que a relação não é uma linha reta simples (ex: a demanda não aumenta linearmente de segunda para domingo). O dia da semana (`Weekday`) é quase certamente um preditor muito importante devido à diferença de padrão entre dias úteis (picos de deslocamento) e fins de semana (uso para lazer). Modelos baseados em árvores (como RandomForest) conseguirão capturar essa relação complexa.

## Resumo e Próximos Passos

1. **Ação Imediata:** Remover a coluna `Dew point temperature(°C)` do dataset para eliminar a redundância.
2. **Preditores Mais Fortes:** Os atributos mais importantes são `Temperature(°C)`, `Hour` e `Solar Radiation (MJ/m2)`.
3. **Preditores Relevantes:** `Rainfall(mm)`, `Snowfall (cm)` e `Humidity(%)` também são importantes e devem ser mantidos.
4. **Não se Engane com Correlação Baixa:** Mantemos `Weekday` e `Hour`. Apesar da correlação linear baixa, eles contêm padrões cíclicos (diários e semanais) que são fundamentais para o problema e serão capturados por modelos mais sofisticados.
5. **Valores não numéricos:**: O algoritmo aprende com números, não podemos passar os atributos da coluna **DateTime**, então devemos excluir essa coluna da nossa base de dados.
6. **Limpeza de instancias (linhas):** Não é necessário fazer nenhuma limpeza de instancias no dataframe, pois analizando como ydata_profiling foi verificado que não existe nenuma linha duplicada, NaN ou em branco.

In [4]:
#removendos as colunas desnecessárias
df_bike = df_bike.drop(['DateTime', 'Dew point temperature(°C)'], axis=1)
#df_bike = df_bike.drop(['DateTime', 'Day', 'Dew point temperature(°C)', 'Weekday', 'Wind speed (m/s)'], axis=1)

print("Dataframe após a remoção das colunas:")
df_bike

Dataframe após a remoção das colunas:


,Day,Weekday,Hour,Rented Bike Count,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
0,12,5,0,254,-5.200,37,2.200,2000,0.000,0.000,0.000
1,12,5,1,204,-5.500,38,0.800,2000,0.000,0.000,0.000
2,12,5,2,173,-6.000,39,1.000,2000,0.000,0.000,0.000
3,12,5,3,107,-6.200,40,0.900,2000,0.000,0.000,0.000
4,12,5,4,78,-6.000,36,2.300,2000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...
8755,31,2,19,163,0.000,31,2.200,2000,0.000,0.000,0.000
8756,31,2,20,161,-1.000,32,0.900,2000,0.000,0.000,0.000
8757,31,2,21,179,-1.600,35,1.000,2000,0.000,0.000,0.000
8758,31,2,22,155,-2.100,36,1.700,2000,0.000,0.000,0.000


# 3. Divisão do dataset
Agora vamos dividir o dataset em três base de dados;
* Treinamento: Para treinar as árvores
* Validação (ou eval_set) para o Early Stopping: Para monitorar a performance e decidir quando parar. 
* Teste: Para a avaliação final.

## Early Stopping
O Early Stopping (Parada Antecipada) é uma técnica de regularização extremamente poderosa e prática, usada para encontrar o número ideal de **n_estimators** e, ao mesmo tempo, prevenir o overfitting. Permite que você defina um n_estimators bem alto e, então, monitora a performance do modelo em um conjunto de dados de validação, parando o treinamento automaticamente quando a performance para de melhorar.  

In [5]:
# Primeiro, dividimos em treino+validação (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(
    df_bike.drop(['Rented Bike Count'], axis=1),   # remove as colunas label 
    df_bike['Rented Bike Count'],   # Coluna que será usada como label de classificação
    test_size=0.2,  # informamos a porcentagem de divisão para a base de testes.
    random_state=35  # aqui informamos um "seed".
)

# Agora, dividimos o conjunto temporário em treino (75% de 80% = 60% do total) 
# e validação (25% de 80% = 20% do total)
X_train_early_stp, X_val, y_train_early_stp, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=35
)

print("Dimensões dos conjuntos:")
print("dataset original", df_bike.shape)
print("X_train_early_stp:", X_train_early_stp.shape, "| y_train_early_stp:", y_train_early_stp.shape)
print("X_val:", X_val.shape, "  | y_val:", y_val.shape)
print("X_test:", X_test.shape, " | y_test:", y_test.shape)
print("-" * 30)

Dimensões dos conjuntos:
dataset original (8760, 11)
X_train_early_stp: (5256, 10) | y_train_early_stp: (5256,)
X_val: (1752, 10)   | y_val: (1752,)
X_test: (1752, 10)  | y_test: (1752,)
------------------------------


# 4. Preparação dos dados

## Remoção de Outliers
Os outliers são prejudiciais para o processo de normalização ou padronização do dataset, então vamos verificar os outliers e remove-los.

1. Identifique os outliers no dataset de treino (usando o método IQR, por exemplo).
2. Defina os limites (limite inferior e superior) com base no dataset de treino.
3. Aplique esses mesmos limites para remover (ou tratar) os outliers no dataset de teste. É crucial usar os limites do treino para todos os conjuntos.

### Por que é Errado Calcular os Limites de Outliers em Cada Conjunto Separadamente?
Isso é uma forma sutil de vazamento de dados (data leakage). Se você calcular os limites do IQR para X_train, e depois recalcular novos limites para X_test, você está usando informações de X_test (sua distribuição, seus quartis) para decidir como processá-lo.

Lembre-se, o conjunto de teste deve ser tratado como se fosse completamente desconhecido até o momento da avaliação final. Você não pode usar nenhuma informação dele para tomar decisões de pré-processamento.

In [6]:
# ---  Tratamento de Outliers ---


# --- IDENTIFICAR AS COLUNAS PARA VERIFICAR OUTLIERS ---
# Vamos focar nas colunas numéricas contínuas onde outliers são mais prováveis e problemáticos.
# Colunas como 'Day', 'Weekday', 'Hour' são mais categóricas/ordinais e geralmente não são tratadas para outliers.
# As colunas Rainfall e Snowfall são compostas com mais de 90% das amostras com valores zerados, oque significa que quase não teve
# dias com chuva ou neve, mas isso pode acontecer, então não são tratadas para outliers
colunas_para_verificar = ['Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 
                          'Visibility (10m)', 'Solar Radiation (MJ/m2)']


# ---  APRENDER OS LIMITES DOS OUTLIERS APENAS COM O CONJUNTO DE TREINO ---
print("--- Aprendendo os limites dos outliers com o conjunto de treino ---")
limites = {}

for coluna in colunas_para_verificar:
    Q1 = X_train[coluna].quantile(0.25)
    Q3 = X_train[coluna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    limites[coluna] = (limite_inferior, limite_superior)
    print(f"Coluna '{coluna}': Limites calculados = [{limite_inferior:.2f}, {limite_superior:.2f}]")

    
# --- APLICAR OS FILTROS A TODOS OS CONJUNTOS DE DADOS ---

# Criar uma cópia para não modificar os dataframes originais
X_train_limpo = X_train.copy()
y_train_limpo = y_train.copy()
X_train_early_stp_limpo = X_train_early_stp.copy()
y_train_early_stp_limpo = y_train_early_stp.copy()
X_val_limpo = X_val.copy()
y_val_limpo = y_val.copy()
X_test_limpo = X_test.copy()
y_test_limpo = y_test.copy()

print("\n--- Removendo outliers ---")
for coluna in colunas_para_verificar:
    limite_inf, limite_sup = limites[coluna]
    
    # Criar máscaras booleanas para cada conjunto de dados
    mascara_train = (X_train_limpo[coluna] >= limite_inf) & (X_train_limpo[coluna] <= limite_sup)
    mascara_train_early_stp = (X_train_early_stp_limpo[coluna] >= limite_inf) & (X_train_early_stp_limpo[coluna] <= limite_sup)
    mascara_val = (X_val_limpo[coluna] >= limite_inf) & (X_val_limpo[coluna] <= limite_sup)
    mascara_test = (X_test_limpo[coluna] >= limite_inf) & (X_test_limpo[coluna] <= limite_sup)
    
    # Aplicar as máscaras para filtrar os dataframes
    X_train_limpo = X_train_limpo[mascara_train]
    y_train_limpo = y_train_limpo[mascara_train]

    X_train_early_stp_limpo = X_train_early_stp_limpo[mascara_train_early_stp]
    y_train_early_stp_limpo = y_train_early_stp_limpo[mascara_train_early_stp]
    
    X_val_limpo = X_val_limpo[mascara_val]
    y_val_limpo = y_val_limpo[mascara_val]
    
    X_test_limpo = X_test_limpo[mascara_test]
    y_test_limpo = y_test_limpo[mascara_test]


# ---  VERIFICAR O RESULTADO ---
print("\n--- Comparação de Tamanhos (Antes vs. Depois) ---")
print(f"Tamanho de X_train: {len(X_train)} -> {len(X_train_limpo)} (removidos {len(X_train) - len(X_train_limpo)} outliers)")
print(f"Tamanho de X_train_early_stp_limpo: {len(X_train_early_stp)} -> {len(X_train_early_stp_limpo)} (removidos {len(X_train_early_stp) - len(X_train_early_stp_limpo)} outliers)")
print(f"Tamanho de X_val:   {len(X_val)} -> {len(X_val_limpo)} (removidos {len(X_val) - len(X_val_limpo)} outliers)")
print(f"Tamanho de X_test:  {len(X_test)} -> {len(X_test_limpo)} (removidos {len(X_test) - len(X_test_limpo)} outliers)")

print("\n--- Amostra do DataFrame de Treino Limpo ---")
display(X_train_limpo.head())

--- Aprendendo os limites dos outliers com o conjunto de treino ---
Coluna 'Temperature(°C)': Limites calculados = [-25.04, 51.06]
Coluna 'Humidity(%)': Limites calculados = [-6.00, 122.00]
Coluna 'Wind speed (m/s)': Limites calculados = [-1.20, 4.40]
Coluna 'Visibility (10m)': Limites calculados = [-637.50, 3582.50]
Coluna 'Solar Radiation (MJ/m2)': Limites calculados = [-1.40, 2.33]

--- Removendo outliers ---

--- Comparação de Tamanhos (Antes vs. Depois) ---
Tamanho de X_train: 7008 -> 6390 (removidos 618 outliers)
Tamanho de X_train_early_stp_limpo: 5256 -> 4786 (removidos 470 outliers)
Tamanho de X_val:   1752 -> 1604 (removidos 148 outliers)
Tamanho de X_test:  1752 -> 1599 (removidos 153 outliers)

--- Amostra do DataFrame de Treino Limpo ---


,Day,Weekday,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
2785,19,5,1,9.300,45,0.800,1263,0.000,0.000,0.000
6921,14,1,9,10.100,60,0.800,726,1.020,0.000,0.000
894,27,7,6,-15.300,47,0.700,1921,0.000,0.000,0.300
320,2,3,8,-8.600,59,1.000,1948,0.010,0.000,1.600
1271,11,1,23,9.700,56,1.300,1423,0.000,0.000,0.000



## Seleção de Atributos
Por que é a melhor abordagem inicial para este dataset?

1. Alta Redundância nos Dados: O dataset possui colunas que são naturalmente correlacionadas e podem carregar informações muito similares. O exemplo mais claro é entre **Temperature(°C)** e **Dew point temperature(°C)** (Temperatura do Ponto de Orvalho). A temperatura do ponto de orvalho é diretamente derivada da temperatura e da umidade. Manter ambas pode ser redundante para muitos modelos. A Seleção de Atributos pode ajudar a identificar e remover uma delas sem grande perda de informação. Alem  de outros atributos que tem baixa correlação com a atributo a ser predito (**Rented Bike Count**).
2. Manutenção da Interpretabilidade: Esta é a maior vantagem. Ao selecionar atributos, você continua trabalhando com as colunas originais (Temperature, Wind speed, Hour, etc.). Isso significa que, ao final, você pode analisar o seu modelo e dizer: "A temperatura e a hora do dia foram os fatores mais importantes para prever a demanda". Essa interpretabilidade é crucial para gerar insights de negócio e entender o "porquê" por trás das previsões.

### VarianceThreshold
Se uma feature (coluna) não varia muito, ela não contém muita informação e, portanto, provavelmente não é útil para um modelo preditivo. Em outras palavras, ele remove todas as features cuja variância não atinge um determinado limiar (threshold). 

Um **threshold=0** significa que será removido apenas as colunas com valores constantes.

In [ ]:
# vamos verificar qual é a variancia de cada coluna
print("\nVariância de cada coluna do dataset de treino:")
print(X_train_limpo.var())
print("-" * 30)

In [14]:
# Aplicar VarianceThreshold
selector = VarianceThreshold(threshold=1) 

# Aprender com o dataset de trein 
X_train_vt_array = selector.fit_transform(X_train_limpo)
# O resultado é um array NumPy. Vamos convertê-lo de volta para um DataFrame
# para melhor visualização, usando os nomes das colunas selecionadas.
colunas = selector.get_feature_names_out()
X_train_vt = pd.DataFrame(data=X_train_vt_array, columns=colunas, index=X_train_limpo.index)

# transformar todos outros dataset com os valores aprendidos com o dataset de treino
X_train_early_stp_vt_array = selector.transform(X_train_early_stp_limpo)
X_train_early_stp_vt = pd.DataFrame(data=X_train_early_stp_vt_array, columns=colunas, index=X_train_early_stp_limpo.index)

X_val_vt_array = selector.transform(X_val_limpo)
X_val_vt = pd.DataFrame(data=X_val_vt_array, columns=colunas, index=X_val_limpo.index)

X_test_vt_array = selector.transform(X_test_limpo)
X_test_vt = pd.DataFrame(data=X_test_vt_array, columns=colunas, index=X_test_limpo.index)

print("Formato de X_train após VarianceThreshold:")
X_train_vt

Formato de X_train após VarianceThreshold:


,Day,Weekday,Hour,Temperature(°C),Humidity(%),Visibility (10m),Rainfall(mm)
2785,19.000,5.000,1.000,9.300,45.000,1263.000,0.000
6921,14.000,1.000,9.000,10.100,60.000,726.000,0.000
894,27.000,7.000,6.000,-15.300,47.000,1921.000,0.000
320,2.000,3.000,8.000,-8.600,59.000,1948.000,0.000
1271,11.000,1.000,23.000,9.700,56.000,1423.000,0.000
...,...,...,...,...,...,...,...
3321,11.000,6.000,9.000,8.900,67.000,578.000,0.000
3007,28.000,7.000,7.000,10.800,78.000,335.000,0.000
7148,23.000,3.000,20.000,13.600,70.000,1392.000,0.000
1295,13.000,3.000,23.000,2.400,59.000,2000.000,0.000


### SelectKBest 
Seleciona um número fixo (K) de features com as maiores pontuações no teste estatístico.

Para Problemas de Regressão:
* **f_regression** (Padrão): Calcula a correlação entre cada feature (X) e a variável alvo (y) e converte isso em um valor-F estatístico. Essencialmente, ele pontua mais alto as features que têm uma relação linear mais forte com o alvo.
* **mutual_info_regression**: Mede a "informação mútua" entre cada feature e o alvo. É mais poderoso que f_regression porque consegue capturar relações não-lineares também.

In [9]:
# Usando a função f_regression do seletor
col = 7
selector_kbest = SelectKBest(score_func=f_regression, k=col)  # vamos escolher as 8 melhores colunas

# Aprender quais são as melhores features com os dados de treino
X_train_kbest_array = selector_kbest.fit_transform(X_train_limpo, y_train_limpo)

# Obter os nomes das colunas mantidas
colunas_selecionadas_nomes = selector_kbest.get_feature_names_out()

# Transformar o X_train_limpo para conter apenas as features selecionadas
X_train_kbest = pd.DataFrame(
    data=X_train_kbest_array, 
    columns=colunas_selecionadas_nomes, 
    index=X_train_limpo.index
)

# Transformar o X_train_early_stp_limpo para conter apenas as features selecionadas
X_train_early_stp_limpo_kbest_array = selector_kbest.transform(X_train_early_stp_limpo)
# Criar o novo DataFrame (esta parte agora funciona corretamente)
X_train_early_stp_kbest = pd.DataFrame(
    data=X_train_early_stp_limpo_kbest_array, 
    columns=colunas_selecionadas_nomes, 
    index=X_train_early_stp_limpo.index
)

# Transformar o X_val_limpo para conter apenas as features selecionadas
X_val_limpo_kbest_array = selector_kbest.transform(X_val_limpo)
# Criar o novo DataFrame (esta parte agora funciona corretamente)
X_val_kbest = pd.DataFrame(
    data=X_val_limpo_kbest_array, 
    columns=colunas_selecionadas_nomes, 
    index=X_val_limpo.index
)

# Transformar o X_test_limpo para conter apenas as features selecionadas
X_test_limpo_kbest_array = selector_kbest.transform(X_test_limpo)
# Criar o novo DataFrame (esta parte agora funciona corretamente)
X_test_kbest = pd.DataFrame(
    data=X_test_limpo_kbest_array, 
    columns=colunas_selecionadas_nomes, 
    index=X_test_limpo.index
)


# ---  VERIFICAR O RESULTADO ---
print("--- DataFrame de Treino após SelectKBest ---")
X_train_kbest

--- DataFrame de Treino após SelectKBest ---


,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Snowfall (cm)
2785,1.000,9.300,45.000,0.800,1263.000,0.000,0.000
6921,9.000,10.100,60.000,0.800,726.000,1.020,0.000
894,6.000,-15.300,47.000,0.700,1921.000,0.000,0.300
320,8.000,-8.600,59.000,1.000,1948.000,0.010,1.600
1271,23.000,9.700,56.000,1.300,1423.000,0.000,0.000
...,...,...,...,...,...,...,...
3321,9.000,8.900,67.000,1.100,578.000,0.690,0.000
3007,7.000,10.800,78.000,0.500,335.000,0.260,0.000
7148,20.000,13.600,70.000,1.400,1392.000,0.000,0.000
1295,23.000,2.400,59.000,3.200,2000.000,0.000,0.000


In [ ]:
# Usando a função f_regression do seletor
col = 7
selector_kbest_mutual = SelectKBest(score_func=mutual_info_regression, k=col)  # vamos escolher as 8 melhores colunas

# Aprender quais são as melhores features com os dados de treino
X_train_kbest_mutual_array = selector_kbest_mutual.fit_transform(X_train_limpo, y_train_limpo)

# Obter os nomes das colunas mantidas
colunas_selecionadas_nomes_mutual = selector_kbest_mutual.get_feature_names_out()

# Transformar o X_train_limpo para conter apenas as features selecionadas
X_train_kbest_mutual = pd.DataFrame(
    data=X_train_kbest_mutual_array, 
    columns=colunas_selecionadas_nomes, 
    index=X_train_limpo.index
)

# Transformar o X_train_early_stp_limpo para conter apenas as features selecionadas
X_train_early_stp_limpo_kbest_mutual_array = selector_kbest_mutual.transform(X_train_early_stp_limpo)
# Criar o novo DataFrame (esta parte agora funciona corretamente)
X_train_early_stp_kbest_mutual = pd.DataFrame(
    data=X_train_early_stp_limpo_kbest_mutual_array, 
    columns=colunas_selecionadas_nomes, 
    index=X_train_early_stp_limpo.index
)

# Transformar o X_val_limpo para conter apenas as features selecionadas
X_val_limpo_kbest_mutual_array = selector_kbest_mutual.transform(X_val_limpo)
# Criar o novo DataFrame (esta parte agora funciona corretamente)
X_val_kbest_mutual = pd.DataFrame(
    data=X_val_limpo_kbest_mutual_array, 
    columns=colunas_selecionadas_nomes, 
    index=X_val_limpo.index
)

# Transformar o X_test_limpo para conter apenas as features selecionadas
X_test_limpo_kbest_mutual_array = selector_kbest_mutual.transform(X_test_limpo)
# Criar o novo DataFrame (esta parte agora funciona corretamente)
X_test_kbest_mutual = pd.DataFrame(
    data=X_test_limpo_kbest_mutual_array, 
    columns=colunas_selecionadas_nomes, 
    index=X_test_limpo.index
)


# ---  VERIFICAR O RESULTADO ---
print("--- DataFrame de Treino após SelectKBest ---")
X_train_kbest_mutual

## Normalização ou padronização
Algoritmos da fámilia Symbolists (Árvores de Decisão, Random Forest, XGBoost, LightGBM...) não são sensíveis à escala das features, portanto não exigem normalização. Eles funcionam fazendo "perguntas" sobre os dados, como "A Temperature é maior que 15°C?". Essa pergunta funciona da mesma forma, não importa se as outras features estão em escalas diferentes. Eles são imunes à escala das features.

Por que não é necessário?

Esses modelos tomam decisões com base em regras ou divisões nos dados, e a escala de uma feature não afeta onde essas divisões são feitas.

Porém, a variância é dependente da escala dos dados. Uma feature salário em Reais terá uma variância muito maior do que uma feature idade em anos, simplesmente por causa da escala. Portanto é uma boa prática sempre normalizar um dataset.

### StandardScaler (Padronização):
**O que faz?** Transforma os dados para que tenham uma média de 0 e um desvio padrão de 1. Isso é chamado de padronização. Os valores resultantes não ficam em um intervalo fixo (podem ser negativos ou positivos, como -1.5, 0.2, 2.1, etc.).

**Quando usar?** É a escolha mais comum e robusta. Funciona bem para a maioria dos algoritmos, especialmente aqueles que assumem que os dados seguem uma distribuição normal (Gaussiana), como a Regressão Linear. É menos sensível a outliers do que o MinMaxScaler.

### MinMaxScaler (Normalização para um Intervalo):
**O que faz?** Transforma os dados para que todos os valores fiquem em um intervalo específico, geralmente entre 0 e 1.

**Quando usar?** É muito útil para algoritmos que esperam dados em um intervalo pequeno e fixo, como Redes Neurais (especialmente com funções de ativação como a sigmoide). Também é bom para visualizações.

**Cuidado:** É muito sensível a outliers. Se você tiver um valor extremo, ele pode "espremer" todos os outros dados em um intervalo muito pequeno, prejudicando a performance.

### Como Aplicar a Normalização Corretamente
A regra de ouro da normalização (e de todo o pré-processamento) é: você deve aprender os parâmetros da normalização (média, desvio padrão, etc.) APENAS no conjunto de treinamento e, em seguida, aplicar essa mesma transformação aos conjuntos de validação e teste.

Isso evita o vazamento de dados (**data leakage**), onde informações do conjunto de teste "vazam" para o processo de treinamento, resultando em uma avaliação de performance otimista e irreal.

In [10]:
# Vamos usar o StandardScaler, que padroniza os dados (média=0, desvio padrão=1).
scaler = StandardScaler()

# --- APRENDER E TRANSFORMAR O CONJUNTO DE TREINO ---
# O scaler "aprende" a média e o desvio padrão de cada coluna do X_train.
# Em seguida, ele usa esses valores para transformar o X_train.
# A função .fit_transform() faz essas duas etapas de uma só vez.
print("Normalizando o conjunto de treinamento...")
#X_train_normalizado_array = scaler.fit_transform(X_train_limpo)
#X_train_normalizado_array = scaler.fit_transform(X_train_vt)
X_train_normalizado_array = scaler.fit_transform(X_train_kbest)


# O resultado é um array NumPy. Vamos convertê-lo de volta para um DataFrame
# para manter os nomes das colunas e a legibilidade.

#X_train_normalizado = pd.DataFrame(X_train_normalizado_array, 
#                                  columns=X_train_limpo.columns, 
#                                  index=X_train_limpo.index)
#X_train_normalizado = pd.DataFrame(X_train_normalizado_array, 
#                                  columns=X_train_vt.columns, 
#                                  index=X_train_vt.index)
X_train_normalizado = pd.DataFrame(X_train_normalizado_array, 
                                  columns=X_train_kbest.columns, 
                                  index=X_train_kbest.index)

# ---  APLICAR A MESMA TRANSFORMAÇÃO NO CONJUNTO DE TESTE ---
# IMPORTANTE: Usamos apenas .transform() aqui.
# Isso garante que o X_test seja normalizado usando a MESMA média e desvio padrão
# que foram aprendidos com o X_train, evitando vazamento de dados (data leakage).
print("Normalizando o conjunto de teste...")
#X_test_normalizado_array = scaler.transform(X_test_limpo)
#X_test_normalizado_array = scaler.transform(X_test_vt)
X_test_normalizado_array = scaler.transform(X_test_kbest)

# Convertendo de volta para um DataFrame
#X_test_normalizado = pd.DataFrame(X_test_normalizado_array, 
#                                 columns=X_test_limpo.columns, 
#                                 index=X_test_limpo.index)
#X_test_normalizado = pd.DataFrame(X_test_normalizado_array, 
#                                 columns=X_test_vt.columns, 
#                                 index=X_test_vt.index)
X_test_normalizado = pd.DataFrame(X_test_normalizado_array, 
                                 columns=X_test_kbest.columns, 
                                 index=X_test_kbest.index)

# ---  APLICAR A MESMA TRANSFORMAÇÃO NO CONJUNTO DE TREINO USADO PARA O Early Stopping ---
# IMPORTANTE: Usamos apenas .transform() aqui.
# Isso garante que o novo dataframe seja normalizado usando a MESMA média e desvio padrão
# que foram aprendidos com o X_train, evitando vazamento de dados (data leakage).
print("Normalizando o conjunto de treinamento usando no metodo Early Stopping...")
#X_train_early_stp_normalizado_array = scaler.transform(X_train_early_stp_limpo)
#X_train_early_stp_normalizado_array = scaler.transform(X_train_early_stp_vt)
X_train_early_stp_normalizado_array = scaler.transform(X_train_early_stp_kbest)

# O resultado é um array NumPy. Vamos convertê-lo de volta para um DataFrame
# para manter os nomes das colunas e a legibilidade.
#X_train_early_stp_normalizado = pd.DataFrame(X_train_early_stp_normalizado_array, 
#                                  columns=X_train_early_stp_limpo.columns, 
#                                  index=X_train_early_stp_limpo.index)
#X_train_early_stp_normalizado = pd.DataFrame(X_train_early_stp_normalizado_array, 
#                                  columns=X_train_early_stp_vt.columns, 
#                                  index=X_train_early_stp_vt.index)
X_train_early_stp_normalizado = pd.DataFrame(X_train_early_stp_normalizado_array, 
                                  columns=X_train_early_stp_kbest.columns, 
                                  index=X_train_early_stp_kbest.index)

# ---  APLICAR A MESMA TRANSFORMAÇÃO NO CONJUNTO DE VALIDAÇÃO USADO PARA O Early Stopping ---
# IMPORTANTE: Usamos apenas .transform() aqui.
# Isso garante que o novo dataframe seja normalizado usando a MESMA média e desvio padrão
# que foram aprendidos com o X_train, evitando vazamento de dados (data leakage).
print("Normalizando o conjunto de validação usado no Early Stopping...")
#X_val_normalizado_array = scaler.transform(X_val_limpo)
#X_val_normalizado_array = scaler.transform(X_val_vt)
X_val_normalizado_array = scaler.transform(X_val_kbest)


# Convertendo de volta para um DataFrame
#X_val_normalizado = pd.DataFrame(X_val_normalizado_array, 
#                                 columns=X_val_limpo.columns, 
#                                 index=X_val_limpo.index)
#X_val_normalizado = pd.DataFrame(X_val_normalizado_array, 
#                                 columns=X_val_vt.columns, 
#                                 index=X_val_vt.index)
X_val_normalizado = pd.DataFrame(X_val_normalizado_array, 
                                 columns=X_val_kbest.columns, 
                                 index=X_val_kbest.index)

print("\nNormalização concluída!")

print("\n--- Amostra do DataFrame de Treino ANTES da Normalização ---")
display(X_train_limpo.head())

print("\n--- Amostra do DataFrame de Treino DEPOIS da Normalização ---")
display(X_train_normalizado.head())

print("\n--- Amostra do DataFrame de Teste ANTES da Normalização ---")
display(X_test_limpo.head())

print("\n--- Amostra do DataFrame de Teste DEPOIS da Normalização ---")
display(X_test_normalizado.head())

print("\n--- Amostra do DataFrame de treino com Early Stopping ANTES da Normalização ---")
display(X_train_early_stp_limpo.head())

print("\n--- Amostra do DataFrame de treino com Early Stopping DEPOIS da Normalização ---")
display(X_train_early_stp_normalizado.head())

print("\n--- Amostra do DataFrame de validação com Early Stopping ANTES da Normalização ---")
display(X_val_limpo.head())

print("\n--- Amostra do DataFrame de validação com Early Stopping DEPOIS da Normalização ---")
display(X_val_normalizado.head())

# Verificando as estatísticas do conjunto de treino normalizado
# A média (mean) deve ser muito próxima de 0 e o desvio padrão (std) próximo de 1.
print("\n--- Estatísticas Descritivas do Conjunto de Treino Normalizado ---")
display(X_train_normalizado.describe().round(2))


Normalizando o conjunto de treinamento...
Normalizando o conjunto de teste...
Normalizando o conjunto de treinamento usando no metodo Early Stopping...
Normalizando o conjunto de validação usado no Early Stopping...

Normalização concluída!

--- Amostra do DataFrame de Treino ANTES da Normalização ---


,Day,Weekday,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
2785,19,5,1,9.300,45,0.800,1263,0.000,0.000,0.000
6921,14,1,9,10.100,60,0.800,726,1.020,0.000,0.000
894,27,7,6,-15.300,47,0.700,1921,0.000,0.000,0.300
320,2,3,8,-8.600,59,1.000,1948,0.010,0.000,1.600
1271,11,1,23,9.700,56,1.300,1423,0.000,0.000,0.000



--- Amostra do DataFrame de Treino DEPOIS da Normalização ---


,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Snowfall (cm)
2785,-1.438,-0.248,-0.745,-0.870,-0.257,-0.625,-0.172
6921,-0.322,-0.180,0.003,-0.870,-1.133,1.002,-0.172
894,-0.741,-2.350,-0.646,-0.977,0.817,-0.625,0.508
320,-0.462,-1.777,-0.047,-0.657,0.861,-0.610,3.457
1271,1.630,-0.214,-0.197,-0.337,0.005,-0.625,-0.172



--- Amostra do DataFrame de Teste ANTES da Normalização ---


...

,Day,Weekday,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
4697,9,2,17,24.500,37,3.200,2000,1.040,0.000,0.000
3478,18,6,22,14.200,73,1.900,2000,0.000,0.000,0.000
1773,5,2,21,11.900,48,1.900,1918,0.000,0.000,0.000
3945,7,5,9,23.300,61,1.700,2000,0.800,0.000,0.000
2327,29,5,23,12.700,75,1.600,475,0.000,0.000,0.000



--- Amostra do DataFrame de validação com Early Stopping DEPOIS da Normalização ---


,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Snowfall (cm)
4697,0.793,1.050,-1.144,1.691,0.946,1.034,-0.172
3478,1.490,0.170,0.651,0.304,0.946,-0.625,-0.172
1773,1.351,-0.026,-0.596,0.304,0.812,-0.625,-0.172
3945,-0.322,0.948,0.053,0.090,0.946,0.651,-0.172
2327,1.630,0.042,0.751,-0.016,-1.542,-0.625,-0.172



--- Estatísticas Descritivas do Conjunto de Treino Normalizado ---


,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Snowfall (cm)
count,6390.000,6390.000,6390.000,6390.000,6390.000,6390.000,6390.000
mean,0.000,0.000,0.000,-0.000,-0.000,-0.000,0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-1.580,-2.490,-2.990,-1.720,-2.270,-0.630,-0.170
25%,-0.880,-0.790,-0.800,-0.760,-0.850,-0.630,-0.170
50%,-0.040,0.030,0.000,-0.230,0.390,-0.630,-0.170
75%,0.930,0.810,0.800,0.620,0.950,0.320,-0.170
max,1.630,2.290,1.900,2.870,0.950,3.080,19.790


# 5. Treinamento dos modelos preditivos

## Regressão
A regressão é usada para prever um valor numérico contínuo, neste caso, o número de bicicletas alugadas.
Como se aplica aqui? Você usa os outros atributos (hora, temperatura, umidade, estação do ano, etc.) como features (variáveis de entrada) para treinar um modelo de regressão. O objetivo do modelo é aprender a relação matemática entre essas features e o Rented Bike Count.

### XGBoost
Vamos usar o algoritmo **XGBoost**


In [12]:
# --- Treinamento com Early Stopping ---

# Instanciar o modelo com um n_estimators alto
xgbr = xgb.XGBRegressor(
    n_estimators=2000, # Um número alto, pois vamos parar antes se necessário. É o número total de especialistas (árvores de decisão)
    learning_rate=0.01, # É o peso dado (de 0 a 1) na confiança que o modelo tem da correção proposta pela nova árvore (n_estimators)
    max_depth=7,   # Uma profundidade de 5 significa que cada árvore pode fazer, no máximo, 3 níveis de perguntas "se-então-senão" para chegar a uma previsão.
    objective='reg:squarederror',   # define matematicamente o que significa "erro" para o seu problema específico. Define a função de perda (loss function) que o algoritmo tentará minimizar durante o treinamento.
    eval_metric='rmse',
    n_jobs=-1,  # Usa todos os núcleos de CPU disponíveis para acelerar o treino
    random_state=35,
    early_stopping_rounds=20 # Paciência de 10 rodadas
)

# Definir o conjunto de validação
eval_set = [(X_train_early_stp_normalizado, y_train_early_stp_limpo), (X_val_normalizado, y_val_limpo)]

print("Iniciando o treinamento com Early Stopping...")
# Treinar o modelo
xgbr.fit(
    X_train_early_stp_normalizado, y_train_early_stp_limpo,
    eval_set=eval_set,
    verbose=False # Mostra o progresso do erro a cada rodada
)
print("Treinamento concluído!")
print(f"Melhor iteração encontrada: {xgbr.best_iteration}")

Iniciando o treinamento com Early Stopping...
Treinamento concluído!
Melhor iteração encontrada: 461


# 6. Analisando os resultados
Agora vamos analisar a predição usando a base de dados para teste.

Vamos usar a metrica RMSE, MAPE, alem de mostar a quantidade bruta de erros na predição.

## RMSE
Ela te dá o erro médio do modelo, na mesma unidade da sua variável alvo. No caso, o resultado do RMSE será em "número de bicicletas".

Exemplo: Um RMSE de 55.8 significa que, em média, as previsões do seu modelo erram por cerca de 56 bicicletas (para mais ou para menos).

Penaliza Erros Grandes: Como ela eleva os erros ao quadrado antes de tirar a média, erros grandes (outliers de previsão) têm um peso muito maior no resultado final. Um único erro grotesco vai inflar bastante o RMSE. Isso é útil porque te alerta sobre previsões muito ruins.

## MAPE 
Um MAPE de 15% significa que, em média, a previsão do seu modelo está a 15% de distância do valor real. Se o valor real era 200 bicicletas, o modelo errou, em média, por 30 bicicletas (15% de 200). Se o valor real era 1000, ele errou por 150.

**Desvantagens e Cuidados (MUITO IMPORTANTE)**

O MAPE tem duas fraquezas significativas que você precisa conhecer:
1. É Indefinido ou Explode com Valores Reais Iguais a Zero:
    * O Problema: Olhe a fórmula: ... / valor_real. Se o valor_real for zero, você terá uma divisão por zero, o que matematicamente é indefinido. No seu dataset, se houver uma hora em que nenhuma bicicleta foi alugada (Rented Bike Count = 0) e seu modelo prever qualquer valor diferente de zero (ex: 10), o cálculo do erro para esse ponto falhará ou resultará em infinito.
    * Consequência: Se o seu y_test contém zeros, a função do Scikit-learn para o MAPE pode retornar um erro ou um valor gigantesco (inf), tornando a métrica inútil.
2. É Assimétrico e Penaliza Mais os Erros de Superestimação:
    * O Problema: O MAPE não trata erros para mais e para menos da mesma forma.
        * Subestimação (prever menos que o real): O erro percentual máximo que você pode ter é 100% (se você prever 0 quando o real era 1000, o erro é (1000-0)/1000 = 100%).
        * Superestimação (prever mais que o real): O erro percentual pode ser infinitamente grande. Se o real era 10 e você previu 1000, o erro é (10-1000)/10 = -99, e o erro percentual é 990%!
    * Consequência: O MAPE "incentiva" os modelos a serem mais conservadores e a preverem valores mais baixos (subestimar), pois os erros de superestimação são penalizados de forma desproporcional.

In [13]:
previsoes_xgbr = xgbr.predict(X_test_normalizado)

# --- Comparar Previsões com Valores Reais ---
# Criar um novo DataFrame para comparar os resultados
resultados = pd.DataFrame({
    'Valor Real': y_test_limpo,
    'Valor Previsto_xgbr': previsoes_xgbr
})

# Adicionar uma coluna com a diferença (erro) para cada previsão
resultados['Erro'] = resultados['Valor Real'] - resultados['Valor Previsto_xgbr']
resultados['Erro %'] = ((resultados['Valor Real'] - resultados['Valor Previsto_xgbr']) * 100) / resultados['Valor Real']

# calculando o rmse entre o valor real e o predito
rmse = mean_absolute_error(y_test_limpo, previsoes_xgbr)
# calculando o mape entre o valor real e o predito
mape = mean_absolute_percentage_error(y_test_limpo, previsoes_xgbr)


# Mostra metricas
print(f"score: {xgbr.score(X_test_normalizado, y_test_limpo)}")
print(f"rmse: {rmse:.2f} bicicletas." )
print(f"mape: {mape:.2f}% de erro medio.")

print("\nAmostra da comparação entre valores reais e previstos:")
display(resultados.head(10))


score: 0.7581508755683899
rmse: 196.11 bicicletas.
mape: 123040608188104704.00% de erro medio.

Amostra da comparação entre valores reais e previstos:


,Valor Real,Valor Previsto_xgbr,Erro,Erro %
2665,513,497.052,15.948,3.109
7650,2200,1733.019,466.981,21.226
6536,2129,975.860,1153.140,54.163
1187,547,611.856,-64.856,-11.857
4244,2519,2175.206,343.794,13.648
4444,123,250.026,-127.026,-103.273
1843,1558,1754.516,-196.516,-12.613
7504,1344,1307.848,36.152,2.690
6596,1835,1302.244,532.756,29.033
3762,3251,2711.093,539.907,16.607


# 7. Conclusões
* Normalizar não trouxe alteração para o modelo xgbr.
* A função **f_regression** e **f_regression** do método **SelectKBest** retornaram as mesmas colunas.
* Usar o SelectKBest manteve as colunas; `Hour, Temperature(°C), Humidity(%), Wind speed (m/s),	Visibility (10m),	Solar Radiation (MJ/m2) e	Snowfall (cm)`
* Usar o VarianceThreshold manteve as colunas; `Day,	Weekday,	Hour,	Temperature(°C),	Humidity(%),	Visibility (10m), e	Rainfall(mm)`
* Usar o VarianceThreshold mostrou melhor resultados doque usar o SelectKBest.
